In [1]:
import pandas as pd
from sklearn import preprocessing

In [2]:
df_train=pd.read_csv("F:/Git/Handeling Categorical Variables/train.csv")
df_test=pd.read_csv("F:/Git/Handeling Categorical Variables/test.csv")

In [3]:
df_test["target"]=-1

In [4]:
df_test.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,600000,0.0,0.0,0.0,F,Y,Blue,Polygon,Axolotl,Finland,...,fced9e114,3.0,Novice,Boiling Hot,f,U,oU,3.0,9.0,-1
1,600001,0.0,0.0,0.0,F,Y,Red,Circle,Lion,Russia,...,7ca8775da,1.0,Novice,Cold,n,N,NaN,2.0,8.0,-1
2,600002,0.0,0.0,0.0,F,Y,Blue,Circle,Axolotl,Russia,...,5940334c9,1.0,Expert,Warm,i,N,DN,2.0,6.0,-1
3,600003,1.0,0.0,0.0,F,N,Red,Polygon,Axolotl,Costa Rica,...,6fbdeefc8,1.0,Expert,Hot,m,B,AG,1.0,6.0,-1
4,600004,0.0,0.0,1.0,F,Y,Red,Circle,NaN,Finland,...,2a27c8fde,1.0,Contributor,Lava Hot,o,J,DT,3.0,3.0,-1


In [10]:
len(df_train.target)

600000

# Label Encoding and One Hot Encoding

In [6]:
class CategoricalFeatures:
    def __init__(self,df,categorical_features,encoding_type):
        self.df=df
        self.output_df=self.df.copy(deep=True)
        self.cat_feats=categorical_features
        self.enc_type=encoding_type
        self.label_encoders=dict()
           
    def _label_encoding(self):
        for c in self.cat_feats:
            lbl=preprocessing.LabelEncoder()
            lbl.fit(self.df[c].values)
            self.output_df.loc[:,c]=lbl.transform(self.df[c].values)
            self.label_encoders[c]=lbl
        return self.output_df
    
    def _one_hot(self):
        ohe = preprocessing.OneHotEncoder()
        ohe.fit(self.df[self.cat_feats].values)
        return ohe.transform(self.df[self.cat_feats].values)
    
    def fit_transform(self):
        if self.enc_type == "label":
            return self._label_encoding()
        elif self.enc_type == "ohe":
            return self._one_hot()
        else:
            raise Exception("Encoding type not understood")
            
    def transform(self, dataframe):
        if self.enc_type == "label":
            for c, lbl in self.label_encoders.items():
                dataframe.loc[:, c] = lbl.transform(dataframe[c].values)
            return dataframe

        elif self.enc_type == "ohe":
            return self.ohe(dataframe[self.cat_feats].values)
        
        else:
            raise Exception("Encoding type not understood")

In [7]:
full_data=pd.concat([df_train,df_test])
train_len=len(df_train)
cols=[c for c in df_train.columns if c not in ["id", "target"]]
print(cols)

['bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4', 'nom_0', 'nom_1', 'nom_2', 'nom_3', 'nom_4', 'nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9', 'ord_0', 'ord_1', 'ord_2', 'ord_3', 'ord_4', 'ord_5', 'day', 'month']


# For OHE first convert the data into label then do one hot encoding

In [8]:
for c in cols:
    full_data.loc[:, c] = full_data.loc[:, c].astype(str).fillna("-9999999")
cat_feats=CategoricalFeatures(full_data,
                              categorical_features=cols,
                              encoding_type="ohe")
full_data_transformed = cat_feats.fit_transform()

In [40]:
train_len

600000

In [19]:
train=full_data_transformed.iloc[:train_len, :]
test=full_data_transformed.iloc[train_len:, :]

In [20]:
train.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0,0,0,0,0,2,4,3,5,...,27,2,0,3,2,20,56,5,5,0
1,1,1,1,0,0,1,2,3,0,6,...,2112,2,2,5,4,23,151,6,9,0
2,2,0,1,0,0,0,2,6,3,0,...,2218,2,5,2,13,15,105,4,11,0
3,3,2,0,0,0,0,2,0,3,3,...,2167,0,4,4,0,2,140,2,5,0
4,4,0,2,0,1,0,2,5,3,2,...,1747,2,2,1,7,2,50,4,3,0


In [32]:
import os
os.getcwd()

'C:\\Users\\Shubham Arrora'

In [12]:
test=df_train.head()
test

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0.0,0.0,0.0,F,N,Red,Trapezoid,Hamster,Russia,...,02e7c8990,3.0,Contributor,Hot,c,U,Pw,6.0,3.0,0
1,1,1.0,1.0,0.0,F,Y,Red,Star,Axolotl,NaN,...,f37df64af,3.0,Grandmaster,Warm,e,X,pE,7.0,7.0,0
2,2,0.0,1.0,0.0,F,N,Red,NaN,Hamster,Canada,...,NaN,3.0,NaN,Freezing,n,P,eN,5.0,9.0,0
3,3,NaN,0.0,0.0,F,N,Red,Circle,Hamster,Finland,...,f9d456e57,1.0,Novice,Lava Hot,a,C,NaN,3.0,3.0,0
4,4,0.0,NaN,0.0,T,N,Red,Triangle,Hamster,Costa Rica,...,c5361037c,3.0,Grandmaster,Cold,h,C,OZ,5.0,12.0,0


In [13]:
test.loc[4, 'id'] = 5

C:\Users\Shubham Arrora\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [14]:
test

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0.0,0.0,0.0,F,N,Red,Trapezoid,Hamster,Russia,...,02e7c8990,3.0,Contributor,Hot,c,U,Pw,6.0,3.0,0
1,1,1.0,1.0,0.0,F,Y,Red,Star,Axolotl,NaN,...,f37df64af,3.0,Grandmaster,Warm,e,X,pE,7.0,7.0,0
2,2,0.0,1.0,0.0,F,N,Red,NaN,Hamster,Canada,...,NaN,3.0,NaN,Freezing,n,P,eN,5.0,9.0,0
3,3,NaN,0.0,0.0,F,N,Red,Circle,Hamster,Finland,...,f9d456e57,1.0,Novice,Lava Hot,a,C,NaN,3.0,3.0,0
4,5,0.0,NaN,0.0,T,N,Red,Triangle,Hamster,Costa Rica,...,c5361037c,3.0,Grandmaster,Cold,h,C,OZ,5.0,12.0,0
